In [ ]:
#setup for processed data
#Note: use for BinaryArray data produced from Entrainment_Preprocessing.ipynb
def GetProcessedString(PROCESSING=False):
    if PROCESSING==True:
        Processed_string="PROCESSED_"
    else:
        Processed_string=""
    return Processed_string

PROCESSING=False 
PROCESSING=True #set to True if using Turbulence-Removed Binary Arrays
Processed_string = GetProcessedString(PROCESSING=PROCESSING)

In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py
from tqdm import tqdm

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
def GetPlottingDirectory(plotFileName, plotType):
    plottingDirectory = mainCodeDirectory=os.path.join(mainDirectory,"Code","PLOTTING")
    
    specificPlottingDirectory = os.path.join(plottingDirectory, plotType, 
                                             f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz")
    os.makedirs(specificPlottingDirectory, exist_ok=True)

    plottingFileName=os.path.join(specificPlottingDirectory, plotFileName)

    return plottingFileName

def SaveFigure(fig,plotType, fileName):
    plotFileName = f"{fileName}_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.jpg"
    plottingFileName = GetPlottingDirectory(plotFileName, plotType)
    print(f"Saving figure to {plottingFileName}")
    fig.savefig(plottingFileName, dpi=300, bbox_inches='tight')

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import *

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracking_Algorithms", dataName="Lagrangian_UpdraftTracking",
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
#data manager class (for saving data)
DataManager_TrackedProfiles = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracked_Profiles", dataName="Tracked_Profiles",
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","2_Tracking_Algorithms"))
from CLASSES_TrackingAlgorithms import TrackingAlgorithms_DataLoading_Class, Results_InputOutput_Class, TrackedParcel_Loading_Class

In [ ]:
# IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","3_Tracked_Profiles"))
from CLASSES_TrackedProfiles import TrackedProfiles_DataLoading_CLASS, TrackedProfiles_Plotting_CLASS, LocationSubset_Plotting_CLASS

In [ ]:
##############################################
#SETUP

In [ ]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
def MakeDataDictionary(variableNames,t,printstatement=False):
    timeString = ModelData.timeStrings[t]
    # print(f"Getting data from {timeString}","\n")
    
    dataDictionary = {variableName: CallLagrangianArray(ModelData, DataManager, timeString, variableName=variableName, printstatement=printstatement) 
                      for variableName in variableNames}      
    return dataDictionary
    
def GetSpatialData(t):    
    variableNames = ['Z']
    dataDictionary = MakeDataDictionary(variableNames,t)
    [Z] = (dataDictionary[k] for k in variableNames)
    return Z

In [ ]:
##############################################
#DATA LOADING

In [ ]:
#Loading in Tracked Parcels Info
trackedArrays,LevelsDictionary = TrackedParcel_Loading_Class.LoadingSubsetParcelData(ModelData,DataManager,
                                                         Results_InputOutput_Class)
hLines,hLineColors = TrackedProfiles_Plotting_CLASS.GetHLines(LevelsDictionary)

In [ ]:
##############################################
#PLOTTING FUNCTIONS FOR ENTRAINMENT

In [ ]:
def ExtraSingleAxisModifications(axes,variableNames):
    #ratio plot vlines
    index = variableNames.index(f'{Processed_string}TransferE_c/{Processed_string}E_c')
    axis = axes[index]
    axis.axvline(0,color='gray',linestyle='dashed',zorder=-15)
    axis.axvline(100,color='gray',linestyle='dashed',zorder=-15)

    #net entrianment zero vlines
    index1 = variableNames.index(f"{Processed_string}E_g-{Processed_string}D_g")
    index2 = variableNames.index(f"{Processed_string}E_c-{Processed_string}D_c")
    for index in [index1,index2]:
        axis = axes[index]
        axis.axvline(0,color='gray',linestyle='dashed',zorder=-15)
    

def ExtraAllAxisModifications(axes,variableNames):
    index = variableNames.index(f'{Processed_string}TransferE_c/{Processed_string}E_c')
    #snap axises to limits
    # SnapLimitsToTicks(axes[:index], dim='x')
    SetEvenTicks(axes[:index], dim='x')
    

    #use scientific notation
    apply_scientific_notation(axes[:index],decimals=2)

    #add hlines
    for axis in axes:
        TrackedProfiles_Plotting_CLASS.PlotHLines(axis, hLines, hLineColors)

In [ ]:
# ============================================================
# TrackedProfiles_Plotting_CLASS
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D

class TrackedProfiles_Plotting_CLASS:

    @staticmethod
    def ProfileMean(profile): 
        """
        Input Requires Three Column Array 
        with Sum in 1st Column, Count in 2nd Column, and Index in 3rd Column
        Returns 1st and 3rd Column (removing zero rows)
        """
        #gets rid of rows that have no data
        profile_mean=profile[ (profile[:, 1] > 1)]; 
        #divides the data column by the counter column
        profile_mean=np.array([profile_mean[:, 0] / profile_mean[:, 1], profile_mean[:, 2]]).T 
        return profile_mean

    # === Category and depth styles ===
    category_styles = {"CL": "solid", "nonCL": "dashed", "SBF": "dashdot"}
    depth_colors = {"SHALLOW": "green", "DEEP": "blue"}

    @staticmethod
    def PlotSE(axis, profile, SE_profile, color, multiplier=1, switch=1, alpha=0.1, min_value=None):
        lower = multiplier * profile[:, 0] - multiplier * SE_profile[:, 0] * switch
        upper = multiplier * profile[:, 0] + multiplier * SE_profile[:, 0] * switch
        
        if min_value is not None:
            lower = np.maximum(lower, min_value)
        axis.fill_betweenx(profile[:, -1], lower, upper, color=color, alpha=alpha)

    @staticmethod
    def GetHLines(LevelsDictionary):
        hLine_1 = LevelsDictionary["min_all_cloudbase"]
        hLine_2 = LevelsDictionary["MeanLFC"]
    
        hLines = (hLine_1,hLine_2)
        hLineColors = ("purple","#FF8C00")
        return hLines,hLineColors

    @staticmethod
    def PlotHLines(axis,hLines,hLineColors):
        for (hLine,hLineColor) in zip(hLines,hLineColors):
            axis.axhline(hLine, color=hLineColor, linestyle='dashed', zorder=-10)

    @staticmethod
    def ApplyXLimFromZLim(axis, zlim, buffer=0.05):
        """
        Adjust the x-limits of the axis by examining all lines plotted on it.
        Only considers x-values where y is within the zlim range.
        """
        x_all = []
        y_all = []
    
        for line in axis.get_lines():
            xdata = line.get_xdata()
            ydata = line.get_ydata()
            x_all.append(xdata)
            y_all.append(ydata)
    
        if not x_all or not y_all:
            return  # No lines to process
    
        x_all = np.concatenate(x_all)
        y_all = np.concatenate(y_all)
    
        mask = (y_all >= zlim[0]) & (y_all <= zlim[1])
        if np.any(mask):
            xmin = np.min(x_all[mask])
            xmax = np.max(x_all[mask])
            delta = xmax - xmin if xmax > xmin else xmax * buffer
            axis.set_xlim(xmin - delta * buffer, xmax + delta * buffer)
    
        axis.set_ylim(zlim)

    @staticmethod
    def AddCategoryLegend(fig, parcelTypes=["CL", "nonCL", "SBF"], loc='upper center', bbox=(0.5, 0.93)):
        """
        Adds a custom legend for parcel types based on linestyle (e.g., CL, nonCL, SBF).
        """
        linestyle_map = {
            "CL": "solid",
            "nonCL": "dashed",
            "SBF": "dashdot"
        }
    
        custom_lines = [
            Line2D([0], [0], color='black', linestyle=linestyle_map[ptype],
                   linewidth=1.5, label=ptype)
            for ptype in parcelTypes if ptype in linestyle_map
        ]
    
        fig.legend(
            handles=custom_lines,
            loc=loc,
            ncol=len(custom_lines),
            fontsize=10,
            title='Parcel Types',
            title_fontsize=12,
            bbox_to_anchor=bbox,
            borderaxespad=0,
            frameon=True
        )

    @staticmethod
    def AddDepthLegend(axis, depths=["ALL", "SHALLOW", "DEEP"]):
        """
        Adds a legend to a specific axis for cloud depth categories (color-coded).
        """
        color_map = {
            "SHALLOW": "green",
            "DEEP": "blue"
        }
    
        legend_lines = [
            Line2D([0], [0], color=color_map[d], linestyle='solid',
                   linewidth=2, label=d)
            for d in depths if d in color_map
        ]
    
        axis.legend(
            handles=legend_lines,
            loc='upper right',
            title='Cloud Types',
            title_fontsize=10,
            fontsize=9,
            frameon=True
        )
    
    # === Level 3: Plot one line ===
    @staticmethod
    def PlotProfileLine(axis, profile, SE_profile, parcelType, parcelDepth,
                        multiplier=1, color=None):
        avg = TrackedProfiles_Plotting_CLASS.ProfileMean(profile)
        x = multiplier * avg[:, 0]
        y = avg[:, 1]
    
        #Allow explicit color override (new behavior)
        color = color or TrackedProfiles_Plotting_CLASS.depth_colors.get(parcelDepth, "gray")
        linestyle = TrackedProfiles_Plotting_CLASS.category_styles.get(parcelType, "solid")
        label = f"{parcelType}-{parcelDepth}"
    
        # Plot main line
        axis.plot(x, y, color=color, linestyle=linestyle, linewidth=1, label=label)
    
        # Plot SE band
        if SE_profile is not None:
            TrackedProfiles_Plotting_CLASS.PlotSE(axis, avg, SE_profile,
                                                  color=color, multiplier=multiplier)
    
    
    # === Level 2: Plot all depths for a given parcelType ===
    @staticmethod
    def PlotAllDepths(axis, profiles, profilesSE, parcelType, variableName,
                      parcelDepths, multiplier=1, color=None, zlim=(0, 6),
                      locationSubset=""):
        for parcelDepth in parcelDepths:
            profile = profiles[parcelType][parcelDepth][variableName][f"profile_array{locationSubset}"]
            SE_profile = None
            if profilesSE:
                SE_profile = profilesSE[parcelType][parcelDepth][variableName].get(f"profile_array{locationSubset}_SE")
    
            #Pass color downstream
            TrackedProfiles_Plotting_CLASS.PlotProfileLine(
                axis, profile, SE_profile, parcelType, parcelDepth,
                multiplier=multiplier, color=color
            )
    
        TrackedProfiles_Plotting_CLASS.ApplyXLimFromZLim(axis, zlim)
    
    
    # === Level 1: Plot one variable to a single axis ===
    @staticmethod
    def PlotSingleVariable(axis, profiles, profilesSE, variableName, variableInfo,
                           parcelTypes, parcelDepths, hLines, hLineColors,
                           color=None, zlim=(0,6),
                           locationSubset=""):
        label = variableInfo[variableName]["label"]
        units = variableInfo[variableName]["units"]
        multiplier = variableInfo[variableName].get("multiplier", 1)
    
        for parcelType in parcelTypes:
            TrackedProfiles_Plotting_CLASS.PlotAllDepths(
                axis, profiles, profilesSE, parcelType, variableName,
                parcelDepths, multiplier=multiplier, color=color, zlim=zlim,
                locationSubset=locationSubset)
            if variableName in ['VMF_g']:
                TrackedProfiles_Plotting_CLASS.PlotAllDepths(
                    axis, profiles, profilesSE, parcelType, "VMF_c",
                    parcelDepths, multiplier=multiplier, color=color, zlim=zlim,
                    locationSubset=locationSubset)
    
        axis.set_ylabel("Height (km)")
        axis.set_xlabel(f"{label} {units}")
        axis.grid(True, linestyle="--", alpha=0.4)
        TrackedProfiles_Plotting_CLASS.PlotHLines(axis, hLines, hLineColors)

    # === Level 1: Plot one variable to a single axis (for operations between multiple variables)
    @staticmethod
    def PlotCompositeVariable(axis, profiles, variableName, variableInfo, 
                              parcelTypes, parcelDepths,
                              color=None, zlim=(0, 6), 
                              printstatement=False,
                              locationSubset=""):
        """
        Plots derived variables defined by multi-step operations in variableInfo['splits'].
        e.g., ["TransferE_c", "-", "TransferE_g", "/", "E_c"]
        """
    
        info = variableInfo[variableName]
        label = info["label"]
        units = info["units"]
        multiplier = info.get("multiplier", 1)
        splits = info.get("splits")
    
        if splits is None:
            raise ValueError(f"'splits' not defined for {variableName}")
    
        for parcelType in parcelTypes:
            for parcelDepth in parcelDepths:
    
                # Load first variable
                first_var = splits[0]
                try:
                    result_prof = profiles[parcelType][parcelDepth][first_var][f"profile_array"]
                    result = TrackedProfiles_Plotting_CLASS.ProfileMean(result_prof)[:, 0]
                    z = TrackedProfiles_Plotting_CLASS.ProfileMean(result_prof)[:, 1]
                except KeyError:
                    print(f"Missing first variable '{first_var}', skipping this combination.")
                    continue
    
                # Apply operations in sequence
                i = 1
                while i < len(splits):
                    op = splits[i]
                    varname = splits[i + 1]
    
                    try:
                        next_prof = TrackedProfiles_Plotting_CLASS.ProfileMean(
                            profiles[parcelType][parcelDepth][varname][f"profile_array"]
                        )[:, 0]
                    except KeyError:
                        next_prof = np.zeros_like(result)
                        print(f"Missing '{varname}', using zeros for '{op}' operation")
    
                    # Perform operation
                    if op == "-":
                        if printstatement==True:
                            print(f"    Performing: ({first_var} - {varname})")
                        result = result - next_prof
                        first_var = f"({first_var}-{varname})"
                    elif op == "/":
                        if printstatement==True:
                            print(f"    Performing: ({first_var} / {varname})")
                        result = np.divide(result, next_prof, out=np.zeros_like(result), where=next_prof != 0)
                        first_var = f"({first_var}/{varname})"
                    else:
                        raise ValueError(f"Unsupported operator '{op}'")
    
                    i += 2  # move to next operator-variable pair
    
                # Apply multiplier
                x = result * multiplier
                y = z
    
                color_use = color or TrackedProfiles_Plotting_CLASS.depth_colors.get(parcelDepth, "gray")
                linestyle = TrackedProfiles_Plotting_CLASS.category_styles.get(parcelType, "solid")
                label_line = f"{parcelType}-{parcelDepth}"
    
                axis.plot(x, y, color=color_use, linestyle=linestyle, linewidth=1, label=label_line)
    
        axis.set_xlabel(f"{label} {units}")
        axis.set_ylabel("Height (km)")
        axis.grid(True, linestyle="--", alpha=0.4)
        TrackedProfiles_Plotting_CLASS.ApplyXLimFromZLim(axis, zlim)

In [ ]:
def PlotAllVariables(profiles, profilesSE, variableNames, variableInfo,
                     parcelTypes, parcelDepths, ncols=2, figsize=(15, 12),
                     locationSubset=""):

    n_vars = len(variableNames)
    nrows = int(np.ceil(n_vars / ncols))

    fig = plt.figure(figsize=figsize)
    gs = gridspec.GridSpec(nrows, ncols, figure=fig, wspace=0.2, hspace=0.35)

    axes = [fig.add_subplot(gs[i // ncols, i % ncols]) for i in range(n_vars)]

    for i, var in enumerate(variableNames):
        axis = axes[i]
        if var == f"{Processed_string}E_g-{Processed_string}D_g":
            zlim=(0,3)
        else:
            zlim=(0,6)
    
        if '-' in var or '/' in var:
            TrackedProfiles_Plotting_CLASS.PlotCompositeVariable(axis, profiles, var, variableInfo, 
                                                                 parcelTypes, parcelDepths, zlim=zlim,
                                                                 locationSubset=locationSubset)
        else:
            TrackedProfiles_Plotting_CLASS.PlotSingleVariable(
                axis, profiles, profilesSE, var, variableInfo,
                parcelTypes, parcelDepths, hLines, hLineColors, zlim=zlim,
                locationSubset=locationSubset)

    # Turn off unused axes (if any)
    for j in range(len(variableNames), nrows * ncols):
        fig.add_subplot(gs[j // ncols, j % ncols]).axis("off")

    TrackedProfiles_Plotting_CLASS.AddCategoryLegend(fig, parcelTypes)
    TrackedProfiles_Plotting_CLASS.AddDepthLegend(axes[0])

    ExtraSingleAxisModifications(axes, variableNames)
    ExtraAllAxisModifications(axes,variableNames)
    return fig

In [ ]:
##############################################
#DATA LOADING FOR ENTRAINMENT

In [ ]:
dataName = f"{Processed_string}Entrainment"
trackedProfileArrays = TrackedProfiles_DataLoading_CLASS.LoadProfile(ModelData,DataManager_TrackedProfiles, dataName, t='combined')

In [ ]:
##############################################
#APPLYING CONSTANTS

In [ ]:
#data manager class
DataManager_Entrainment = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="EntrainmentCalculation", dataName="EntrainmentCalculation",
                                dtype='int32')

#getting entrainment constant data
entrainmentConstant = DataManager_Entrainment.LoadCalculations(
        DataManager_Entrainment.outputDataDirectory,
        dataName="EntrainmentConstant",
        verbose=False,
    )["entrainmentConstant"]

In [ ]:
def ApplyEntrainmentConstant(trackedProfileArrays,entrainmentConstant):
    for key1 in trackedProfileArrays:  # e.g. 'CL', 'nonCL', etc.
        for key2 in trackedProfileArrays[key1]:  # e.g. 'ALL', 'SHALLOW', 'DEEP'
            for key3 in trackedProfileArrays[key1][key2]:  # e.g. 'D_c', 'E_g', etc.
                
                for arrayName in ["profile_array","profile_array_left","profile_array_right"]:
                    arr = trackedProfileArrays[key1][key2][key3][arrayName]
                    arr[:, 0] *= entrainmentConstant
                for arrayName in ["profile_array_squares","profile_array_left_squares","profile_array_right_squares"]:
                    arr_SE = trackedProfileArrays[key1][key2][key3][arrayName]
                    arr_SE[:, 0] *= entrainmentConstant**2
    return trackedProfileArrays

In [ ]:
def FixDetrainmentNegative(trackedProfileArrays):
    for key1 in trackedProfileArrays:  # e.g. 'CL', 'nonCL', etc.
        for key2 in trackedProfileArrays[key1]:  # e.g. 'ALL', 'SHALLOW', 'DEEP'
            for key3 in trackedProfileArrays[key1][key2]:  # e.g. 'D_c', 'E_g', etc.
                if (f'{Processed_string}D' in key3) and (f'{Processed_string}Transfer' not in key3):
                    for arrayName in ["profile_array","profile_array_left","profile_array_right"]:
                        arr = trackedProfileArrays[key1][key2][key3][arrayName]
                        arr[:, 0] *= -1
    return trackedProfileArrays

In [ ]:
trackedProfileArrays = ApplyEntrainmentConstant(trackedProfileArrays,entrainmentConstant)
trackedProfileArrays = FixDetrainmentNegative(trackedProfileArrays)

In [ ]:
#finally making SE profiles
trackedProfiles_SE = TrackedProfiles_DataLoading_CLASS.ExtractProfileStandardErrorArrays(trackedProfileArrays,ProfileStandardError)

In [ ]:
##############################################
#PLOTTING SETUP FOR ENTRAINMENT

In [ ]:
variableInfo = {
    f"{Processed_string}E_g-{Processed_string}D_g": { 
        "label": r"$(E-D)_g$",
        "units": r"($kg\ m^{-3}\ s^{-1}$)",
        "multiplier": 1,
        "splits": [f"{Processed_string}E_g", "-", f"{Processed_string}D_g"]
    }, 
    f"{Processed_string}E_c-{Processed_string}D_c": { 
        "label": r"$(E-D)_c$",
        "units": r"($kg\ m^{-3}\ s^{-1}$)",
        "multiplier": 1,
        "splits": [f"{Processed_string}E_c", "-", f"{Processed_string}D_c"]
    }, 
    f"{Processed_string}TransferE_c": { 
        "label": r"$E_{g\rightarrow c}$",
        "units": r"($kg\ m^{-3}\ s^{-1}$)",
        "multiplier": 1,
        "splits": [f"{Processed_string}TransferE_c", "-", f"{Processed_string}TransferE_g"]
    }, 
    f"{Processed_string}TransferE_c/{Processed_string}E_c": { 
        "label": r"$E_{g\rightarrow c}\ /\ E_c$",
        "units": r"$(\%)$",
        "multiplier": 100,
        "splits": [f"{Processed_string}TransferE_c", "/", f"{Processed_string}E_c"]
    }, 
}

In [ ]:
##############################################
#PLOTTING FOR ENTRAINMENT

In [ ]:
variableNames = [f"{Processed_string}E_g-{Processed_string}D_g", f"{Processed_string}E_c-{Processed_string}D_c",
                 f"{Processed_string}TransferE_c", f"{Processed_string}TransferE_c/{Processed_string}E_c"]

fig = PlotAllVariables(
    profiles=trackedProfileArrays,
    profilesSE=trackedProfiles_SE,
    variableNames=variableNames,
    variableInfo=variableInfo,
    parcelTypes=["CL", "nonCL", "SBF"],
    parcelDepths=["SHALLOW", "DEEP"],
    locationSubset="")

#saving
fileName=f"Tracked_Profiles_{dataName}" 
SaveFigure(fig,plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_{dataName}",fileName=fileName)

In [ ]:
##############################################
#PLOTTING FUNCTIONS FOR LOCATION SUBSETTING

In [ ]:
def ApplyFormatting(fig, variableNames):
    axes = fig.get_axes()
    SetEvenTicks(axes)
    apply_scientific_notation(axes, decimals=1)

    # Match each variable’s axes across parcel types
    for variableName in variableNames:
        variableAxes = LocationSubset_Plotting_CLASS.GetVariableAxes(fig, variableName)
        MatchAxisLimits_V2(variableAxes)

    # Reset z-lim for general 
    variableNameList = [variableName for variableName in variableNames if "_g" in variableName]
    for variableName in variableNameList:
        variableAxes = LocationSubset_Plotting_CLASS.GetVariableAxes(fig, variableName)
        for axis in variableAxes:
            axis.set_ylim((0,3))

In [ ]:
##############################################
#PLOTTING FOR LOCATION SUBSETTING

In [ ]:
variableNames = [f"{Processed_string}E_g-{Processed_string}D_g", f"{Processed_string}E_c-{Processed_string}D_c",
                 f"{Processed_string}TransferE_c", f"{Processed_string}TransferE_c/{Processed_string}E_c"]

fig = PlotAllVariables(
    profiles=trackedProfileArrays,
    profilesSE=trackedProfiles_SE,
    variableNames=variableNames,
    variableInfo=variableInfo,
    parcelTypes=["CL", "nonCL", "SBF"],
    parcelDepths=["SHALLOW", "DEEP"],
    locationSubset="_left")

#saving
fileName=f"Tracked_Profiles_{dataName}_LEFT" 
SaveFigure(fig,plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_{dataName}",fileName=fileName)

In [ ]:
variableNames = [f"{Processed_string}E_g-{Processed_string}D_g", f"{Processed_string}E_c-{Processed_string}D_c",
                 f"{Processed_string}TransferE_c", f"{Processed_string}TransferE_c/{Processed_string}E_c"]

fig = PlotAllVariables(
    profiles=trackedProfileArrays,
    profilesSE=trackedProfiles_SE,
    variableNames=variableNames,
    variableInfo=variableInfo,
    parcelTypes=["CL", "nonCL", "SBF"],
    parcelDepths=["SHALLOW", "DEEP"],
    locationSubset="_right")

#saving
fileName=f"Tracked_Profiles_{dataName}_RIGHT" 
SaveFigure(fig,plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_{dataName}",fileName=fileName)

In [ ]:
variableNames = [f"{Processed_string}E_g-{Processed_string}D_g", f"{Processed_string}E_c-{Processed_string}D_c"]

# variableNames=[f"{Processed_string}E_g",f"{Processed_string}E_c"]


# fig = LocationSubset_Plotting_CLASS.PlotProfiles(
fig = LocationSubset_Plotting_CLASS.PlotCompositeProfiles(
    trackedProfileArrays,
    variableInfo,
    variableNames = variableNames,
    ncols_inner = 1,
    top_adjust=0.75
)
ApplyFormatting(fig, variableNames)


#saving
fileName=f"Tracked_Profiles_{dataName}_LocationSubset" 
SaveFigure(fig,plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_{dataName}",fileName=fileName)